In [4]:
import pandas as pd
import numpy as np

hrs = pd.read_csv("life_expectancy_CleanedHRSdata.csv")
pd.set_option('display.max_columns', None)

print(f'Shape of data: {hrs.shape} \n \n Columns of data: {hrs.columns}')
hrs

Shape of data: (4711, 55) 
 
 Columns of data: Index(['Unnamed: 0', 'hhidpn', 'wave', 'mstat', 'cendiv', 'gender', 'rahispan',
       'raracem', 'iwbeg', 'dage_m', 'dage_y', 'raedyrs', 'rarelig', 'ravetrn',
       'agey_m', 'shlt', 'shltc', 'depres', 'effort', 'sleepr', 'cesd', 'bmi',
       'smokev', 'smoken', 'drinkn', 'hibp', 'diab', 'cancr', 'lung', 'heart',
       'strok', 'psych', 'arthr', 'conde', 'cogtot', 'slfmem', 'pstmem',
       'spcfac', 'hsptim', 'puff', 'puffpos', 'timwlk', 'hatotb', 'iearn',
       'isret', 'covs', 'hiltc', 'lbrf', 'logiearn', 'logisret', 'loghspti',
       'loghatotb', 'id', 'nt', 'n2'],
      dtype='object')


,Unnamed: 0,hhidpn,wave,mstat,cendiv,gender,rahispan,raracem,iwbeg,dage_m,dage_y,raedyrs,rarelig,ravetrn,agey_m,shlt,shltc,depres,effort,sleepr,cesd,bmi,smokev,smoken,drinkn,hibp,diab,cancr,lung,heart,strok,psych,arthr,conde,cogtot,slfmem,pstmem,spcfac,hsptim,puff,puffpos,timwlk,hatotb,iearn,isret,covs,hiltc,lbrf,logiearn,logisret,loghspti,loghatotb,id,nt,n2
0,1,3010,3,1.married,9.pacific,1.male,0.not hispanic,1.white/caucasian,13345,931,77,12,1.protestant,0.no,60,3.good,-1,1.yes,1.yes,1.yes,3,28.000000,0.no,0.no,1,0.no,0.no,0.no,0.no,1.yes,0.no,0.no,0.no,1,35,4.fair,2.same,0.no,0,NaN,NaN,NaN,490500.0,4000.0,0.0,1.yes,1.yes,4.partly retired,8.294049,0.000000,0.0,13.103181,2,123702,7
1,2,3010,6,1.married,9.pacific,1.male,0.not hispanic,1.white/caucasian,15445,931,77,12,1.protestant,0.no,66,3.good,-1,0.no,0.no,0.no,1,28.299999,0.no,0.no,0,0.no,0.no,0.no,0.no,1.yes,0.no,0.no,0.no,1,31,3.good,2.same,0.no,0,NaN,NaN,NaN,704000.0,10000.0,13728.0,0.no,1.yes,4.partly retired,9.210340,9.527193,0.0,13.464534,3,123702,7
2,3,3010,7,1.married,9.pacific,1.male,0.not hispanic,1.white/caucasian,16267,931,77,12,1.protestant,0.no,68,3.good,0,0.no,0.no,0.no,0,26.600000,0.no,0.no,0,0.no,0.no,0.no,0.no,1.yes,0.no,0.no,0.no,1,18,3.good,2.same,0.no,0,NaN,NaN,NaN,756000.0,6000.0,15600.0,0.no,1.yes,4.partly retired,8.699514,9.655026,0.0,13.535797,4,123702,7
3,4,3010,8,1.married,9.pacific,1.male,0.not hispanic,1.white/caucasian,16875,931,77,12,1.protestant,0.no,70,3.good,0,0.no,0.no,0.no,0,27.100000,0.no,0.no,0,0.no,0.no,0.no,0.no,1.yes,0.no,0.no,0.no,1,20,4.fair,2.same,0.no,0,490.0,1.standing,2.78,914000.0,0.0,14040.0,0.no,1.yes,5.retired,0.000000,9.549665,0.0,13.725586,5,123702,7
4,6,3010,10,1.married,9.pacific,1.male,0.not hispanic,1.white/caucasian,18520,931,77,12,1.protestant,0.no,74,3.good,0,0.no,0.no,0.no,0,24.000000,0.no,0.no,0,0.no,0.no,0.no,0.no,1.yes,0.no,0.no,0.no,1,17,4.fair,3.worse,0.no,0,330.0,1.standing,3.10,1240000.0,0.0,16644.0,0.no,1.yes,5.retired,0.000000,9.719805,0.0,14.030622,7,123702,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4706,13126,213435010,4,7.widowed,5.s atlantic,2.female,0.not hispanic,1.white/caucasian,14045,1082,90,12,2.catholic,0.no,74,3.good,.p,0.no,0.no,0.no,0,24.900000,1.yes,0.no,2,1.yes,0.no,0.no,0.no,0.no,0.no,0.no,0.no,1,24,3.good,2.same,0.no,0,NaN,NaN,NaN,87000.0,0.0,8856.0,0.no,1.yes,5.retired,0.000000,9.088850,0.0,11.373663,107639,123702,7
4707,13127,213435010,5,7.widowed,5.s atlantic,2.female,0.not hispanic,1.white/caucasian,14715,1082,90,12,2.catholic,0.no,75,3.good,0,0.no,0.no,0.no,0,25.700001,1.yes,0.no,2,1.yes,0.no,0.no,0.no,0.no,0.no,0.no,0.no,1,24,3.good,2.same,0.no,0,NaN,NaN,NaN,90000.0,0.0,8400.0,0.no,1.yes,5.retired,0.000000,9.035987,0.0,11.407565,107640,123702,7
4708,13128,213435010,6,7.widowed,5.s atlantic,2.female,0.not hispanic,1.white/caucasian,15475,1082,90,12,2.catholic,0.no,78,3.good,0,0.no,0.no,0.no,0,27.100000,1.yes,0.no,2,1.yes,0.no,0.no,0.no,0.no,0.no,0.no,0.no,1,23,3.good,3.worse,0.no,1,NaN,NaN,NaN,84750.0,0.0,9600.0,0.no,1.yes,5.retired,0.000000,9.169519,0.0,11.347461,107641,123702,7
4709,13129,213435010,7,7.widowed,5.s atlantic,2.female,0.not hispanic,1.white/caucasian,16206,1082,90,12,2.catholic,0.no,80,3.good,0,0.no,0.no,0.no,0,25.400000,1.yes,0.no,3,1.yes,0.no,0.no,0.no,0.no,0.no,0.no,1.yes,2,25,3.good,3.worse,0.no,1,NaN,NaN,NaN,111000.0,0.0,9912.0,0.no,1.yes,5.retired,0.000000,9.201502,0.0,11.617286,107642,123702,7


In [2]:
### These are functions from the raw_data_processing.ipynb
def standardize_dataframe(df, numerical_columns, exclude_columns):
    """
    A function to standardize specified numerical values in a DataFrame using z-score normalization,
    excluding specified columns.
    
    Parameters:
        df (pandas DataFrame): The input DataFrame.
        numerical_columns (list): A list of numerical column names to standardize.
        exclude_columns (list): A list of column names to exclude from standardization.
        
    Returns:
        pandas DataFrame: A DataFrame with standardized numerical values.
    """
    # Exclude columns specified in exclude_columns
    columns_to_standardize = [col for col in numerical_columns if col not in exclude_columns]
    
    # Initialize the StandardScaler
    scaler = StandardScaler()
    
    # Fit the scaler to the selected columns and transform the values
    standardized_values = scaler.fit_transform(df[columns_to_standardize])
    
    # Create a new DataFrame with the standardized values and the same index and columns as the original DataFrame
    standardized_df = pd.DataFrame(standardized_values, index=df.index, columns=columns_to_standardize)
    
    # Combine the standardized numerical columns with non-numerical columns from the original DataFrame
    for col in df.columns:
        if col not in columns_to_standardize:
            standardized_df[col] = df[col]
    
    return standardized_df

def encode_categorical(df, categorical_vars):
    """
    A function to perform one-hot encoding for categorical variables in a DataFrame.
    
    Parameters:
        df (pandas DataFrame): The input DataFrame.
        categorical_columns (list): A list of column names containing categorical variables to be one-hot encoded.
        
    Returns:
        pandas DataFrame: A DataFrame with one-hot encoded categorical variables.
    """
    print(f"encodable categorical vars: {categorical_vars}")
    
    # Convert numeric categorical variables to categorical type
    for col in categorical_vars:
        df[col] = df[col].astype('category')
        
    # Extract categorical variables
    categorical_df = df[categorical_vars]
    
    # Perform one-hot encoding for categorical variables, drop first ensures there is no multicolinearity
    result = pd.get_dummies(categorical_df, dtype=float, drop_first=True)
    
    return result

def replace_encoded_categorical(df, encoded_categorical_df, categorical_columns):
    """
    A function to replace original categorical columns in a DataFrame with one-hot encoded columns.
    
    Parameters:
        df (pandas DataFrame): The original DataFrame.
        encoded_categorical_df (pandas DataFrame): The DataFrame with one-hot encoded categorical variables.
        categorical_columns (list): A list of column names containing original categorical variables.
        
    Returns:
        pandas DataFrame: A DataFrame with original categorical columns replaced by one-hot encoded columns.
    """
    # Drop original categorical columns from the original DataFrame
    df = df.drop(columns=categorical_columns)
    
    # Concatenate the original DataFrame with the one-hot encoded DataFrame
    df = pd.concat([df, encoded_categorical_df], axis=1)
    return df

In [3]:
#columns_to_drop = ['Unnamed: 0', 'nt','n2','dage_y', 'rarelig']
columns_to_drop = ['Unnamed: 0', 'nt','n2','dage_y', 'rarelig']
hrs_cleaned = hrs.drop(columns=columns_to_drop, errors='ignore')
    # Replace empty strings with NaN to handle both empty strings and NaN values uniformly
hrs_cleaned = hrs_cleaned.replace('', np.nan)
    
    # Drop rows with any NaN values
hrs_cleaned = hrs_cleaned.dropna()
def random_sample_per_group(df, group_col):
    """
    Randomly selects one observation per group from a DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame to sample from.
    group_col (str): The name of the column to group by.

    Returns:
    pd.DataFrame: A DataFrame with one randomly selected row per group.
    """
    # Group by the specified column and apply the sampling
    return df.groupby(group_col).apply(lambda x: x.sample(1)).reset_index(drop=True)

def is_categorical(df):
    """
    Determines which columns in a DataFrame are categorical based on data type being a string.
    
    Parameters:
        df (pandas DataFrame): The DataFrame to check.
        
    Returns:
        list: A list of column names that are considered categorical because they are of string type.
    """
    categorical_vars = [col for col in df.columns if df[col].dtype == 'object']
    return categorical_vars

# Determine categorical columns
categorical_columns = is_categorical(hrs_cleaned)  # Adjust threshold as necessary
print("Categorical columns:", categorical_columns)
# To remove columns
categorical_columns = [col for col in categorical_columns if col not in ['raedyrs']]
# Encode categorical variables
encoded_df = encode_categorical(hrs_cleaned.copy(), categorical_columns)

# Replace original categorical columns with encoded ones
final_df = replace_encoded_categorical(hrs_cleaned, encoded_df, categorical_columns)

# Convert all values to integers, setting '17.17+ yrs' specifically to 17
final_df['raedyrs'] = final_df['raedyrs'].replace('17.17+ yrs', '17').astype(int)

# Create a binary indicator for whether the education years are 17 and above
final_df['raedyrs_17plus'] = (final_df['raedyrs'] >= 17).astype(int)

# Check to make sure each "hhidpn" appears only once
def random_sample_per_group(df, group_col):
    """
    Randomly selects one observation per group from a DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame to sample from.
    group_col (str): The name of the column to group by.

    Returns:
    pd.DataFrame: A DataFrame with one randomly selected row per group.
    """
    # Group by the specified column and apply the sampling
    return df.groupby(group_col).apply(lambda x: x.sample(1)).reset_index(drop=True)

# Randomly sample one observation per individual
final_df = random_sample_per_group(final_df, 'hhidpn')

# Check to make sure each "hhidpn" appears only once
def check_unique_hhidpn(sampled_df, group_col):
    """
    Checks if each group identifier appears only once in the DataFrame.

    Parameters:
    sampled_df (pd.DataFrame): The DataFrame to check.
    group_col (str): The name of the column to group by.
    """
    if sampled_df[group_col].duplicated().any():
        print("Some hhidpn values appear more than once.")
    else:
        print("Each hhidpn value appears only once.")

# Perform the check
check_unique_hhidpn(final_df, 'hhidpn')

columns_to_drop = ['hhidpn', 'iwbeg', 'id']
final_df = final_df.drop(columns=columns_to_drop, errors='ignore')


# Shuffle the columns randomly
shuffled_columns = np.random.permutation(final_df.columns)

# Split the columns approximately in half
midpoint = len(shuffled_columns) // 2
first_half_columns = shuffled_columns[:midpoint]
second_half_columns = shuffled_columns[midpoint:]

# Create two new DataFrames based on these split columns
df_first_half = final_df[first_half_columns]
df_second_half = final_df[second_half_columns]

# Print out the columns to verify
print("First half columns:", df_first_half.columns)
print("Second half columns:", df_second_half.columns)

Categorical columns: ['mstat', 'cendiv', 'gender', 'rahispan', 'raracem', 'raedyrs', 'ravetrn', 'shlt', 'shltc', 'depres', 'effort', 'sleepr', 'smokev', 'smoken', 'hibp', 'diab', 'cancr', 'lung', 'heart', 'strok', 'psych', 'arthr', 'slfmem', 'pstmem', 'spcfac', 'puffpos', 'covs', 'hiltc', 'lbrf']
encodable categorical vars: ['mstat', 'cendiv', 'gender', 'rahispan', 'raracem', 'ravetrn', 'shlt', 'shltc', 'depres', 'effort', 'sleepr', 'smokev', 'smoken', 'hibp', 'diab', 'cancr', 'lung', 'heart', 'strok', 'psych', 'arthr', 'slfmem', 'pstmem', 'spcfac', 'puffpos', 'covs', 'hiltc', 'lbrf']
Each hhidpn value appears only once.
First half columns: Index(['shlt_2.very good', 'gender_2.female', 'raedyrs', 'lbrf_3.unemployed',
       'sleepr_1.yes', 'shlt_3.good', 'hibp_1.yes', 'wave', 'slfmem_4.fair',
       'mstat_2.married,spouse absent', 'depres_1.yes', 'cendiv_3.en central',
       'mstat_4.separated', 'covs_1.yes', 'loghatotb', 'conde', 'dage_m',
       'raracem_3.other', 'shlt_5.poor', 'p

In [56]:
# Convert all values to integers, setting '17.17+ yrs' specifically to 17
final_df['raedyrs'] = final_df['raedyrs'].replace('17.17+ yrs', '17').astype(int)

# Create a binary indicator for whether the education years are 17 and above
final_df['raedyrs_17plus'] = (final_df['raedyrs'] >= 17).astype(int)

# Print to verify changes
final_df

,wave,dage_m,raedyrs,agey_m,cesd,bmi,drinkn,conde,cogtot,hsptim,puff,timwlk,hatotb,iearn,isret,logiearn,logisret,loghspti,loghatotb,"mstat_2.married,spouse absent",mstat_3.partnered,mstat_4.separated,mstat_5.divorced,mstat_7.widowed,mstat_8.never married,cendiv_2.mid atlantic,cendiv_3.en central,cendiv_4.wn central,cendiv_5.s atlantic,cendiv_6.es central,cendiv_7.ws central,cendiv_8.mountain,cendiv_9.pacific,gender_2.female,rahispan_1.hispanic,raracem_2.black/african american,raracem_3.other,ravetrn_1.yes,shlt_2.very good,shlt_3.good,shlt_4.fair,shlt_5.poor,shltc_-2,shltc_-3,shltc_-4,shltc_0,shltc_1,shltc_2,shltc_3,shltc_4,depres_1.yes,effort_1.yes,sleepr_1.yes,smokev_1.yes,smoken_1.yes,hibp_1.yes,diab_1.yes,cancr_1.yes,lung_1.yes,lung_4.disp prev record and no cond,heart_1.yes,heart_4.disp prev record and no cond,strok_1.yes,strok_2.tia/possible stroke,strok_4.disp prev record and no cond,psych_1.yes,psych_4.disp prev record and no cond,arthr_1.yes,arthr_4.disp prev record and no cond,slfmem_2.very good,slfmem_3.good,slfmem_4.fair,slfmem_5.poor,pstmem_2.same,pstmem_3.worse,spcfac_0.no,spcfac_1.yes,puffpos_2.sitting,covs_0.no,covs_1.yes,hiltc_0.no,hiltc_1.yes,lbrf_2.works pt,lbrf_3.unemployed,lbrf_4.partly retired,lbrf_5.retired,lbrf_6.disabled,lbrf_7.not in lbrf,raedyrs_17plus
0,10,931,12,74,0,24.000000,0,1,17,0,330.0,3.10,1240000.0,0.0,16644.0,0.0,9.719805,0.0,14.030622,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,10,925,16,72,0,25.700001,0,3,25,1,370.0,2.47,1240000.0,0.0,8400.0,0.0,9.035987,0.0,14.030622,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
2,10,965,9,78,4,20.500000,0,1,11,1,150.0,6.50,0.0,0.0,10140.0,0.0,9.224243,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
3,11,954,9,77,2,19.600000,4,3,18,0,340.0,3.83,0.0,0.0,9900.0,0.0,9.200290,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
4,10,973,11,73,2,21.100000,0,1,15,0,400.0,4.19,32999.0,0.0,6540.0,0.0,8.785692,0.0,10.404233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,8,1035,17,76,0,27.400000,1,2,22,0,560.0,4.43,840000.0,0.0,0.0,0.0,0.000000,0.0,13.641157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1
686,9,1093,16,83,1,25.799999,0,1,14,1,180.0,4.06,400000.0,0.0,14400.0,0.0,9.574984,0.0,12.899220,0.0,0.0,0.0,0.0,0.0,0.0,

In [8]:
def random_sample_per_group(df, group_col):
    """
    Randomly selects one observation per group from a DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame to sample from.
    group_col (str): The name of the column to group by.

    Returns:
    pd.DataFrame: A DataFrame with one randomly selected row per group.
    """
    # Group by the specified column and apply the sampling
    return df.groupby(group_col).apply(lambda x: x.sample(1)).reset_index(drop=True)

# Randomly sample one observation per individual
final_df = random_sample_per_group(final_df, 'hhidpn')

# Check to make sure each "hhidpn" appears only once
def check_unique_hhidpn(sampled_df, group_col):
    """
    Checks if each group identifier appears only once in the DataFrame.

    Parameters:
    sampled_df (pd.DataFrame): The DataFrame to check.
    group_col (str): The name of the column to group by.
    """
    if sampled_df[group_col].duplicated().any():
        print("Some hhidpn values appear more than once.")
    else:
        print("Each hhidpn value appears only once.")

# Perform the check
check_unique_hhidpn(final_df, 'hhidpn')

Each hhidpn value appears only once.


In [9]:
final_df

,hhidpn,wave,iwbeg,dage_m,raedyrs,agey_m,cesd,bmi,drinkn,conde,cogtot,hsptim,puff,timwlk,hatotb,iearn,isret,logiearn,logisret,loghspti,loghatotb,id,"mstat_2.married,spouse absent",mstat_3.partnered,mstat_4.separated,mstat_5.divorced,mstat_7.widowed,mstat_8.never married,cendiv_2.mid atlantic,cendiv_3.en central,cendiv_4.wn central,cendiv_5.s atlantic,cendiv_6.es central,cendiv_7.ws central,cendiv_8.mountain,cendiv_9.pacific,gender_2.female,rahispan_1.hispanic,raracem_2.black/african american,raracem_3.other,ravetrn_1.yes,shlt_2.very good,shlt_3.good,shlt_4.fair,shlt_5.poor,shltc_-2,shltc_-3,shltc_-4,shltc_0,shltc_1,shltc_2,shltc_3,shltc_4,depres_1.yes,effort_1.yes,sleepr_1.yes,smokev_1.yes,smoken_1.yes,hibp_1.yes,diab_1.yes,cancr_1.yes,lung_1.yes,lung_4.disp prev record and no cond,heart_1.yes,heart_4.disp prev record and no cond,strok_1.yes,strok_2.tia/possible stroke,strok_4.disp prev record and no cond,psych_1.yes,psych_4.disp prev record and no cond,arthr_1.yes,arthr_4.disp prev record and no cond,slfmem_2.very good,slfmem_3.good,slfmem_4.fair,slfmem_5.poor,pstmem_2.same,pstmem_3.worse,spcfac_0.no,spcfac_1.yes,puffpos_2.sitting,covs_0.no,covs_1.yes,hiltc_0.no,hiltc_1.yes,lbrf_2.works pt,lbrf_3.unemployed,lbrf_4.partly retired,lbrf_5.retired,lbrf_6.disabled,lbrf_7.not in lbrf,raedyrs_17plus
0,3010,8,16875,931,12,70,0,27.100000,0,1,20,0,490.0,2.78,914000.0,0.0,14040.0,0.0,9.549665,0.0,13.725586,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,3020,10,18520,925,16,72,0,25.700001,0,3,25,1,370.0,2.47,1240000.0,0.0,8400.0,0.0,9.035987,0.0,14.030622,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
2,11277010,10,18701,965,9,78,4,20.500000,0,1,11,1,150.0,6.50,0.0,0.0,10140.0,0.0,9.224243,0.0,0.000000,724,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
3,11477010,9,17637,954,9,73,0,22.400000,1,3,18,0,430.0,3.86,0.0,0.0,13200.0,0.0,9.487972,0.0,0.000000,888,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
4,11841010,10,18489,973,11,73,2,21.100000,0,1,15,0,400.0,4.19,32999.0,0.0,6540.0,0.0,8.785692,0.0,10.404233,1138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,213377010,8,16967,1035,17,76,0,27.400000,1,2,22,0,560.0,4.43,840000.0,0.0,0.0,0.0,0.000000,0.0,13.641157,107353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1